In [2]:
import PIL, random, sys
from PIL import Image, ImageDraw
import numpy as np

# Trying to prototype a cellular automaton based map generator.
I'm going to try generating large, random rooms using some combination of procedural generation algorithms. 

As a first step, I'll try a simple cellular automata style iterative generator.

My general thought process is :
1. Initiate a grid object of fixed dimensions
2. Place Tile objects at each tile spot of the grid, randomly instantiated as passable (floor) or impassable (wall)
3. Use the Grid object to check each tile and calculate whether it should mutate based on some neighborhood metrics, set a flag in the tile object if it should mutate
4. Mutate the tiles!

#### 1) Classes
First, I define a Tile class, which will simply keep track of whether the Tile is a wall or a floor. The Tile class will have a mutate function to flip that assignment as well as a flag to indicate if the tile should mutate in the next iteration.

In [94]:
FLOOR_COLOR = (183, 245, 145)
WALL_COLOR = (88, 117, 69)

        
class Tile:
    def __init__(self, p_wall):
        self.will_mutate = False
        
        if np.random.sample() <= p_wall:
            self.isWall = True
        else:
            self.isWall = False
        self.set_color()

    
    def mutate(self):
        if self.mutate:
            self.isWall = not self.isWall
            self.will_mutate = False
            self.set_color()
            
    def set_color(self):
        if self.isWall:
            self.color = WALL_COLOR
        else:
            self.color = FLOOR_COLOR

In [105]:
class MapGrid:
    def __init__(self, nrows, ncols, imwidth, p_wall):
        #get the dimensions of the image
        self.imwidth = imwidth
        self.ncols = ncols
        self.nrows = nrows
        self.tile_size = imwidth/ncols
        self.imheight = tile_size * nrows
        self.p_wall = p_wall
        #initiate the grid of tiles
        self.grid = []
        for i in range(nrows):
            arr = []
            for j in range(ncols):
                arr.append(Tile(self.p_wall))
            self.grid.append(arr)
    
    #The outer rim of the map should all be walls for now
    def make_outline(self):
        for tile in self.grid[0]:
            tile.isWall = True
            tile.set_color()
        for tile in self.grid[-1]:
            tile.isWall = True
            tile.set_color()
        for row in self.grid[1:-1]:
            row[0].isWall = True
            row[0].set_color()
            row[-1].isWall = True
            row[-1].set_color()
    
    def flag_mutations(self):
        #don't bother trying to mutate edges
        for i, row in enumerate(self.grid):
            if i == 0 or i == len(self.grid)-1:
                continue
            for j, tile in enumerate(row):
                if j == 0 or j == len(row)-1:
                    continue
                wall_neighbors = 0
                floor_neighbors = 0
                #Should I store this in the Tile object?
                cardinals = ((i-1, j-1), (i-1, j), (i-1, j+1),
                             (i, j-1), (i, j+1),
                             (i+1, j-1), (i+1, j), (i+1, j+1))
                for cardinal in cardinals:
                    if self.grid[cardinal[0]][cardinal[1]].isWall:
                        wall_neighbors += 1
                    else:
                        floor_neighbors += 1
                
                #CONDITIONAL MUTATION, TRY MESSING WITH THIS
    
                if tile.isWall and wall_neighbors <= 3:
                    tile.will_mutate = True
                
                elif not tile.isWall and wall_neighbors > 3:
                    tile.will_mutate = True
    

                    
    def cause_mutations(self):       
        for row in self.grid:
            for tile in row:
                if tile.will_mutate:
                    tile.mutate()

In [106]:
P_WALL = 0.4

imwidth = 800
ncols = 200
nrows = 200



mapGrid = MapGrid(nrows, ncols, imwidth, P_WALL)
mapGrid.make_outline()

iters = 8
im = Image.new('RGB', (image_dim, image_dim))
pen = ImageDraw.Draw(im)   

for y, row in enumerate(mapGrid.grid):
    for x, tile in enumerate(row):
        x0 = x*grid.tile_size
        x1 = x0 + grid.tile_size
        y0 = y*grid.tile_size
        y1 = y0 + grid.tile_size
        pen.rectangle((x0,y0,x1,y1), tile.color)

im.save('MapIters/map_iteration{0}.png'.format(0))
for i in range(iters):
    mapGrid.flag_mutations()
    mapGrid.cause_mutations()
    im = Image.new('RGB', (image_dim, image_dim))
    pen = ImageDraw.Draw(im)   
    
    for y, row in enumerate(mapGrid.grid):
        for x, tile in enumerate(row):
            x0 = x*grid.tile_size
            x1 = x0 + grid.tile_size
            y0 = y*grid.tile_size
            y1 = y0 + grid.tile_size
            pen.rectangle((x0,y0,x1,y1), tile.color)

    im.save('MapIters/map_iteration{0}.png'.format(i+1))

# Drunkard's Walk 

# Below this is the space invader code, for reference

In [7]:
import PIL, random, sys
from PIL import Image, ImageDraw

r = lambda: random.randint(50,215)
rc = lambda: (r(), r(), r())

def symmetric_list(elements, length):
    l = [] 
    q = []
    loops = int(length/2)
    for i in range(loops):
        choice = random.choice(elements)
        l.append(choice)
        q.append(choice)
    
    if length%2 == 1:
        l.append(random.choice(elements))
        
    for i in q[::-1]:
        l.append(i)
    return l



def draw_creature(bounds, pen, n_pixels):
    x0, y0, x1, y1 = bounds
    pixel_size = (x1-x0)/n_pixels
    colors = [rc(), rc(), rc(), (0,0,0), (0,0,0), (0,0,0)]
    for y in range(0, n_pixels):
        draw_colors = symmetric_list(colors, n_pixels)
        for x, color in enumerate(draw_colors):
            top_left_x = x * pixel_size + x0
            top_left_y = y * pixel_size + y0
            bot_right_x = top_left_x + pixel_size
            bot_right_y = top_left_y + pixel_size
            pen.rectangle((top_left_x, top_left_y, bot_right_x, bot_right_y), color)

            
def main(creature_width, n_creatures, image_dim, show=True, save=None):            
    creature_size = image_dim/n_creatures
    padding = creature_size/creature_width

    im = Image.new('RGB', (image_dim, image_dim))
    pen = ImageDraw.Draw(im)            
    for x in range(0, n_creatures):
        for y in range(0, n_creatures):
            x0 = x*creature_size + padding/2
            y0 = y*creature_size + padding/2
            x1 = x0 + creature_size - padding
            y1 = y0 + creature_size - padding
            draw_creature((x0,y0,x1,y1), pen, creature_width)
    
    if save:
        im.save(save)
    if show:
        im.show()

In [28]:
main(10, 10, 1000, show=True, save='Examples/invaders_10x10.png')

In [38]:
#construct the list of squares in the top quadrant
#  Similar to sym list.

n_pixels = 100
image_dim = 1000
pixel_size = image_dim/n_pixels

im = Image.new('RGB', (image_dim, image_dim))
pen = ImageDraw.Draw(im)   

def make_row(n):
    row = []
    q = []
    for x in range(int(n/2)):
        color = random.choice([rc(), (0,0,0)]) #either a random rgb or black
        row.append(color)                                  
        q.append(color)
    if n%2 == 1:
        row.append(random.choice([rc(), (0,0,0)]))
    for i in q[::-1]:
        row.append(i)
    return(row)

rows = []
for y in range(int(n_pixels/2)):
    rows.append(make_row(n_pixels))
row_q = rows[::-1]
if n_pixels%2 == 1:
    rows.append(make_row())
for row in row_q:
    rows.append(row)
    
for y, row in enumerate(rows):
    for x, color in enumerate(row):
        x0 = x*pixel_size
        x1 = x0 + pixel_size
        y0 = y*pixel_size
        y1 = y0 + pixel_size
        pen.rectangle((x0,y0,x1,y1), color)
im.show()

In [7]:
import pandas as pd

In [47]:
imwidth = 400
imheight = 2*imwidth
im = Image.new('RGB', (imwidth, imheight))
pen = ImageDraw.Draw(im)   

linesize = 3
padding = (0.05 * imwidth)
n_cols = int((imwidth - (2*padding)) / (2*linesize))
n_rows = int(n_cols/2)

point_records = []
starty = padding
startx = padding
for i in range(n_rows):
    record = {}
    for j in range(n_cols):
        x = startx + j*2*linesize
        x = x + np.random.normal(0,1)
        y = starty + i*2*linesize
        y = y + np.random.normal(0,1)
        record["column{}".format(j)] = (x,y)
    point_records.append(record)
    
df = pd.DataFrame.from_records(point_records)

for row in df.values:
    for point in row:
        pen.point(point)
        
for col in df.columns:
    column = df[col]
    for i in range(column.shape[0] - 1):
        p1 = column[i]
        p2 = column[i+1]
        pen.line(xy=[p1,p2],width=1)
        
im.show()

In [41]:
p1

'c'